In [1]:
from yucca.image_processing.transforms.Spatial import Spatial
from yucca.image_processing.transforms.BiasField import BiasField
from yucca.image_processing.transforms.Noise import AdditiveNoise, MultiplicativeNoise
from yucca.image_processing.transforms.Ghosting import MotionGhosting
from yucca.image_processing.transforms.Ringing import GibbsRinging
from nibabel.testing import data_path
import nibabel as nib
import os
from copy import deepcopy
import numpy as np
import timeit

np.random.seed(4215235)
iterations = 2
# We use a publicly available sample from Nibabel
example_file = os.path.join(data_path, 'example4d.nii.gz')
im = nib.load(example_file)
imarr = im.get_fdata()[:, :, :, 0]

# If you want to use a local example for more realistic times, replace the imarr variable
# My local variable:
local_file = '/home/zcr545/YuccaData/yucca_raw_data/Task001_TestTask/imagesTr/TestTask_1000_000.nii.gz'
im = nib.load(local_file)
imarr = im.get_fdata()

# Convert it to the format expected of the transforms
# a dict of {"image": image, "seg": segmentation}
# with samples for dims (b, c, x, y, z) for 3D or (b, c, x, y) for 2D
imarr = imarr[np.newaxis, np.newaxis]
seg = np.zeros(imarr.shape)
datadict = {"image": imarr, "seg": seg}
print("Data is shaped :", imarr.shape)

Data is shaped : (1, 1, 256, 287, 256)


In [2]:
idx = np.array(imarr.shape[2:])//4
imarr_half = imarr[:, :, idx[0]:-idx[0], idx[1]:-idx[1], idx[2]:-idx[2]]
seg_half = np.zeros(imarr_half.shape)
datadict_half = {"image": imarr_half, "seg": seg_half}

In [3]:
# Applying Rotation
tform = Spatial(do_rot=True,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45))
print("Rotation: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)

Rotation:  8.736863824000011 
  Rotation half:  1.0467968265002128


In [4]:
# Applying Deformation
from yucca.image_processing.transforms.Spatial import Spatial

tform = Spatial(do_deform=True,
                deform_alpha=(700, 700),
                deform_sigma=(10, 10))
print("Deformation: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Deformation half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Deformation:  18.89154750749958 
  Deformation half:  2.274235193000095


In [5]:
# Applying Scaling
tform = Spatial(do_scale=True,
                scale_factor=(1.5, 1.5))
print("Scaling: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Scaling half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)

Scaling:  5.013034512499871 
  Scaling half:  0.638087225000163


In [6]:
# Applying Rotation, Deformation and Scaling WITHOUT cropping
tform = Spatial(do_rot=True,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                do_deform=True,
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                do_scale=True,
                scale_factor=(1.5, 1.5))
print("Rotation, Deformation and Scaling WITHOUT cropping: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation, Deformation and Scaling WITHOUT cropping half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Rotation, Deformation and Scaling WITHOUT cropping:  14.831546043000344 
  Rotation, Deformation and Scaling WITHOUT cropping half:  1.8724256274999789


In [7]:
# Applying Rotation, Deformation and Scaling WITH cropping
tform = Spatial(do_crop=True,
                patch_size=np.array(imarr.shape[2:])//2,
                do_rot=True,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                do_deform=True,
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                do_scale=True,
                scale_factor=(1.5, 1.5))
# Applying Rotation, Deformation and Scaling WITH cropping
tform2 = Spatial(do_crop=True,
                patch_size=np.array(imarr_half.shape[2:])//2,
                do_rot=True,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                do_deform=True,
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                do_scale=True,
                scale_factor=(1.5, 1.5))
print("Rotation, Deformation and Scaling WITH cropping: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation, Deformation and Scaling WITH cropping half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Rotation, Deformation and Scaling WITH cropping:  3.245525150499816 
  Rotation, Deformation and Scaling WITH cropping half:  1.8999758925001515


In [8]:
# Applying Additive Noise
tform = AdditiveNoise(p_per_sample=1, mean=(0, 0),
                      sigma=(25, 25))
print("Additive Noise: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Additive Noise half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Additive Noise:  1.4583239190001223 
  Additive Noise half:  0.168274997500248


In [9]:

# Applying Multiplicative Noise
tform = MultiplicativeNoise(p_per_sample=1, mean=(0, 0),
                            sigma=(0.1, 0.1))
print("Multiplicative Noise: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Multiplicative Noise half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Multiplicative Noise:  1.7251039340003445 
  Multiplicative Noise half:  0.19161714600022606


In [10]:
# Applying Bias Field
tform = BiasField(p_per_sample=1)
print("Bias Field: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Bias Field half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Bias Field:  6.7102291085002435 
  Bias Field half:  0.5645473719996517


In [11]:
# Applying Motion Ghosting
tform = MotionGhosting(p_per_sample=1, alpha=(0.6, 0.6),
                       numReps=(3, 4),
                       dims=(0, 1))
print("Motion Ghosting: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Motion Ghosting half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Motion Ghosting:  3.656702298500022 
  Motion Ghosting half:  0.4397559460003322


In [12]:

# Applying Gibbs Ringing
tform = GibbsRinging(p_per_sample=1, cutFreq=(30, 36),
                     dim=(0, 1))
print("Gibbs Ringing: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Gibbs Ringing half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Gibbs Ringing:  3.764839685500192 
  Gibbs Ringing half:  0.47306435049995343
